# redhat_predict

# import

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import os
from collections import deque
from collections import Counter
import codecs
import copy
from sklearn.metrics import confusion_matrix
import sys
from platform import python_version
from datetime import datetime
import time
import shutil
from sklearn.preprocessing import OneHotEncoder
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

# environment

In [ ]:
print(python_version())

In [ ]:
sys.getdefaultencoding()

In [ ]:
%autosave 0

In [ ]:
#最大表示列数の指定（ここでは2000列を指定）
pd.set_option('display.max_columns', 2000)

In [ ]:
#最大表示行数の指定（ここでは200行を指定）
pd.set_option('display.max_rows', 50)

# define

In [ ]:
# n_estimators:ここを増やすと精度は上がるが遅い。。。
ESTIMATOR_NUM=10
RANDOM_STATE_VAL = 0
target_col_name = "outcome"
id_col_name = "activity_id"
drop_columns = "date"

# function

In [ ]:
def dispalyDf(readDf, dispFlag, row_num, dispLabel):
    print("========================= {} =========================".format(dispLabel))
    row_num , col_num = np.shape(readDf)
    print("row_num = {}, col_num = {}".format(row_num, col_num))
    if dispFlag:
        display(readDf.head(row_num))
    print("++++++ None Count All++++++")
    print(readDf.isnull().values.sum())
    print("++++++ None Count ++++++")
    print(readDf.isnull().sum())
    print("++++++ Col Type ++++++")
    print(readDf.dtypes)
    print("++++++ Unique By Col（Noneは含まないので注意） ++++++")
    for col_name in readDf.columns.tolist():
        print("{} : {}".format(col_name, readDf[col_name].nunique()))

# process

In [ ]:
act_train_read_Df = pd.read_csv('input/act_train.csv')
act_test_read_Df = pd.read_csv('input/act_test.csv')
people_read_Df = pd.read_csv('input/people.csv')

In [ ]:
act_train_Df = act_train_read_Df.copy()
act_test_Df = act_test_read_Df.copy()
people_Df = people_read_Df.copy()
del act_train_read_Df, act_test_read_Df, people_read_Df

In [ ]:
dispalyDf(act_train_Df, True, 10, "act_train_Df")

In [ ]:
dispalyDf(act_test_Df, True, 10, "act_test_Df")

In [ ]:
dispalyDf(people_Df, True, 10, "people_Df")

In [ ]:
################################################

In [ ]:
# train:データのDataFrameと、ターゲットのデータフレームに分ける。
train_target_Ser = act_train_Df[target_col_name]
act_train_Df = act_train_Df.loc[:,[col_name for col_name in act_train_Df.columns.tolist() if col_name not in [id_col_name,target_col_name]]]
print(set(train_target_Ser))

In [ ]:
# test:idとデータのDataFrameを抽出。
test_id_Df = act_test_Df.loc[:,[id_col_name]]
act_test_Df = act_test_Df.loc[:,[col_name for col_name in act_test_Df.columns.tolist() if col_name != id_col_name]]

In [ ]:
###### 前処理 ######
def getBeroreProcessDf(input_Df, drop_columns):
    retDf = input_Df.copy()
    retDf = retDf.drop(drop_columns, axis=1)
    
    retDf['people_id'] = retDf['people_id'].apply(lambda x: x.split('_')[1])

    columns = [col_name for col_name in retDf.columns.tolist() if col_name != 'people_id']
    
    for col_name in columns:
        if retDf[col_name].dtype == object:
            # 文字列のカラムの加工処理
            retDf[col_name] = retDf[col_name].fillna('type 0')
            retDf[col_name] = retDf[col_name].apply(lambda x: x.split(' ')[1])
            
        # int64に変換する。
        retDf[col_name] = pd.to_numeric(retDf[col_name])
    return retDf
    

In [ ]:
act_train_Df_2 = getBeroreProcessDf(act_train_Df, drop_columns)

In [ ]:
dispalyDf(act_train_Df_2, True, 10, "act_train_Df_2")

In [ ]:
act_test_Df_2 = getBeroreProcessDf(act_test_Df, drop_columns)

In [ ]:
dispalyDf(act_test_Df_2, True, 10, "act_test_Df_2")

In [ ]:
people_Df_2 = getBeroreProcessDf(people_Df, drop_columns)

In [ ]:
dispalyDf(people_Df_2, True, 10, "people_Df_2")

In [ ]:
train_features = act_train_Df_2.merge(people_Df_2, how='left', on='people_id')

In [ ]:
dispalyDf(train_features, True, 10, "train_features")

In [ ]:
test_features = act_test_Df_2.merge(people_Df_2, how='left', on='people_id')

In [ ]:
dispalyDf(test_features, True, 10, "test_features")

In [ ]:
# train-test間でカラムが一致しているかチェック
if train_features.columns.tolist() != train_features.columns.tolist():
    raise Exception("columns Not Match!!")

In [ ]:
# hold-out
X_train, X_test, y_train, y_test = train_test_split(train_features, train_target_Ser, test_size=0.2, random_state=RANDOM_STATE_VAL)

In [ ]:
#### 学習 ####
ETC = ExtraTreesClassifier()
RFC = RandomForestClassifier(n_estimators=ESTIMATOR_NUM, random_state=RANDOM_STATE_VAL)
DTC = DecisionTreeClassifier()
LOG = LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL,max_iter=120,
                        solver="liblinear")


# 既にある程度高い精度を持つ異なるタイプの学習器（たとえばRandomForest、LogisticRegressionと
# GausianNB(ナイーブベース分類器)など）の出した結果を多数決もしくは確率平均で決めるというもの。
VCLASS = VotingClassifier(
    estimators=[
        ('etc',ETC),
        ('rfc',RFC),
        ('dtc',DTC),
        ('log',LOG),
               ],
    voting='soft')
pipeline = Pipeline([
                     ('MinMaxScaler', MinMaxScaler()),   #MinMaxScaler: 正規化（最大1, 最小0）
                     ('VotingClassifier', VCLASS)
])
                     
pipeline.fit(X_train, y_train)

In [ ]:
pred_score = roc_auc_score(y_test,
                           pipeline.predict_proba(X_test)[:,1])
print("auc : {}".format(pred_score))

In [ ]:
pred_score = f1_score(y_test,
                           pipeline.predict(X_test), pos_label=1)
print("f1 : {}".format(pred_score))

In [ ]:
#### 予測 ####
test_proba = pipeline.predict_proba(test_features)
print(test_proba[:10])
target_Df = pd.DataFrame(
    {
        target_col_name: test_proba[:,1] 
    })

In [ ]:
output_Df = test_id_Df.join(target_Df)

In [ ]:
dispalyDf(output_Df, True, 10, "output_Df")

In [ ]:
# ディレクトリなければ作る
output_dir_path = os.path.join(os.getcwd(), 'output')
print(output_dir_path)
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path) 

In [ ]:
# 出力
output_Df.to_csv('output/submission.csv',
                 sep=",",
                 header=True,
                 index=False,
                 mode="w",
                 encoding="utf-8",
                 line_terminator="\n"
)

In [ ]:
########################################################

In [ ]:
raise Exception("END♪")

In [ ]:
########################################################